## Setup

In [ ]:
!pip install keras-tuner --upgrade
import numpy as np
import tensorflow as tf
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, mean_absolute_error, make_scorer
from kerastuner.tuners import Hyperband
from tensorflow.keras.optimizers import Adam, SGD, RMSprop
from tensorflow.keras.losses import MeanSquaredError, MeanAbsoluteError, Huber, LogCosh
from tensorflow.keras.layers import Dense, Dropout, Lambda, InputLayer
from tensorflow.keras.models import Sequential, load_model
from tensorflow.math import sin, cos
import wandb
from wandb.integration.keras import WandbCallback
import matplotlib.pyplot as plt
from sklearn.svm import SVR
from sklearn.multioutput import MultiOutputRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import RandomizedSearchCV

In [ ]:
wandb.login()

In [ ]:
seed = 2193258

In [ ]:
def start_log_wandb(lr, model_name, dataset, epochs, opt, loss, metrics, batch_size, split, hidden_layers, units_per_layer, dropout, full_output, noise, noise_percent):
  wandb.init(
    project="MLH1",

    config={
    "learning_rate": lr,
    "architecture": model_name,
    "dataset": dataset,
    "epochs": epochs,
    "optimizer": opt,
    "loss": loss,
    "metrics": metrics,
    "batch_size": batch_size,
    "split": split,
    "hidden_layers": hidden_layers,
    "units_per_layer": units_per_layer,
    "dropout": dropout,
    "full_output": full_output,
    "noise": noise,
    "noise_percent": noise_percent
    }
  )

In [ ]:
if tf.config.list_physical_devices('GPU'):
  print("GPU is available.")
  tf.config.set_visible_devices([], 'GPU')
else:
  print("GPU is not available.")

## Files

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
base_path = '/content/drive/MyDrive/MLH1/'

In [ ]:
d2_file = base_path + 'datasetR2.csv'
d3_file = base_path + 'datasetR3.csv'
d5_file = base_path + 'datasetR5.csv'

## Dataset


### Dataset Hyperparameters

In [ ]:
split = 0.2
full_output = False
use_trig = False
noise = False
noise_percent = 10
neural_network = True

In [ ]:
def add_noise_to_dataset(dataframe, noise_percentage):
    random_generator = np.random.default_rng(seed)
    noise_standard_deviation = 0.01
    noisy_dataframe = dataframe.copy()

    number_of_noisy_rows = int((noise_percentage / 100) * len(noisy_dataframe))

    indices_for_noisy_rows = random_generator.choice(
        noisy_dataframe.index, size=number_of_noisy_rows, replace=False
    )

    for column in noisy_dataframe.select_dtypes(include=np.number).columns:
        noisy_dataframe.loc[indices_for_noisy_rows, column] += random_generator.normal(
            0, noise_standard_deviation, size=number_of_noisy_rows
        )

    noisy_dataframe = noisy_dataframe.round(3)
    return noisy_dataframe



### Dataset R2

In [ ]:
df2 = pd.read_csv(d2_file, delimiter=';')
if noise:
  df2 = generate_dataset(df2, noise_percent)

if full_output:
  df_target2 = df2[[' ee_x', ' ee_y', ' ee_qw', ' ee_qz']].copy()
  df_target2.columns = ['ee_x', 'ee_y', 'ee_qw', 'ee_qz']
else:
  df_target2 = df2[[' ee_x', ' ee_y']].copy()
  df_target2.columns = ['ee_x', 'ee_y']
df2.drop([' cos(j0)', ' cos(j1)', ' sin(j0)', ' sin(j1)', ' ee_x', ' ee_y', ' ee_qw', ' ee_qz'], axis=1, inplace=True)

X_train2, X_test2, y_train2, y_test2 = train_test_split(df2, df_target2, test_size=split, random_state=seed)

if neural_network:
  X_train = tf.convert_to_tensor(X_train2, dtype=tf.float32)
  y_train = tf.convert_to_tensor(y_train2, dtype=tf.float32)

  X_test = tf.convert_to_tensor(X_test2, dtype=tf.float32)
  y_test = tf.convert_to_tensor(y_test2, dtype=tf.float32)
else:
  X_train = np.array(X_train2)
  y_train = np.array(y_train2)

  X_test = np.array(X_test2)
  y_test = np.array(y_test2)
dataset = 'r2'

### Dataset R3


In [ ]:
df3 = pd.read_csv(d3_file, delimiter=';')
if noise:
  df3 = generate_dataset(df3, noise_percent)

if full_output:
  df_target3 = df3[[' ee_x', ' ee_y', ' ee_qw', ' ee_qz']].copy()
  df_target3.columns = ['ee_x', 'ee_y', 'ee_qw', 'ee_qz']
else:
  df_target3 = df3[[' ee_x', ' ee_y']].copy()
  df_target3.columns = ['ee_x', 'ee_y']
df3.drop([' cos(j0)', ' cos(j1)', ' cos(j2)', ' sin(j0)', ' sin(j1)',' sin(j2)',' ee_x', ' ee_y', ' ee_qw', ' ee_qz'], axis=1, inplace=True)

X_train3, X_test3, y_train3, y_test3 = train_test_split(df3, df_target3, test_size=split, random_state=seed)

if neural_network:
  X_train = tf.convert_to_tensor(X_train3, dtype=tf.float32)
  y_train = tf.convert_to_tensor(y_train3, dtype=tf.float32)

  X_test = tf.convert_to_tensor(X_test3, dtype=tf.float32)
  y_test = tf.convert_to_tensor(y_test3, dtype=tf.float32)
else:
  X_train = np.array(X_train3)
  y_train = np.array(y_train3)

  X_test = np.array(X_test3)
  y_test = np.array(y_test3)

dataset = 'r3'

### Dataset R5

In [ ]:
df5 = pd.read_csv(d5_file, delimiter=';')

if noise:
    df5 = generate_dataset(df5, noise_percent)

df_target5 = df5[[' ee_x', ' ee_y', ' ee_z']].copy()

df5 = df5[['j0', ' j1', ' j2', ' j3', ' j4']].copy()

df_target5.columns = ['ee_x', 'ee_y', 'ee_z']

X_train5, X_test5, y_train5, y_test5 = train_test_split(df5, df_target5, test_size=split, random_state=seed)

if neural_network:
    X_train = tf.convert_to_tensor(X_train5, dtype=tf.float32)
    y_train = tf.convert_to_tensor(y_train5, dtype=tf.float32)

    X_test = tf.convert_to_tensor(X_test5, dtype=tf.float32)
    y_test = tf.convert_to_tensor(y_test5, dtype=tf.float32)
else:
    X_train = np.array(X_train5)
    y_train = np.array(y_train5)

    X_test = np.array(X_test5)
    y_test = np.array(y_test5)

dataset = 'r5'

## Forward Kinematics

#### Hyperparameters

In [ ]:
learning_rate = 0.00025
loss = 'mse'
metrics = ['mae']
epochs = 200
batch_size = 32
opt = 'adam'
split = 0.2
hidden_layers = 5
units_per_layer = 80
dropout = 0
print(dataset)

r5


In [ ]:
if opt == 'adam':
  optimizer = Adam(learning_rate=learning_rate)
elif opt == 'sgd':
  optimizer = SGD(learning_rate=learning_rate)
elif opt == 'RMSprop':
  optimizer = RMSprop(learning_rate=learning_rate)

input_dim = X_train.shape[1]
output_dim = y_train.shape[1]

#### Model

In [ ]:
model = Sequential()
model.add(InputLayer(shape=(input_dim,)))
r5_units = [224, 96, 192, 160, 96, 32, 160, 160]
if use_trig:
  if dataset == 'r2':
    model.add(Lambda(lambda inputs: tf.stack([inputs[:, 0], inputs[:, 1],
                                                sin(inputs[:, 0]), sin(inputs[:, 1]),
                                                cos(inputs[:, 0]), cos(inputs[:, 1])], axis=1)))
  elif dataset == 'r3':
    model.add(Lambda(lambda inputs: tf.stack([inputs[:, 0], inputs[:, 1], inputs[:, 2],
                                                sin(inputs[:, 0]), sin(inputs[:, 1]), sin(inputs[:, 2]),
                                                cos(inputs[:, 0]), cos(inputs[:, 1]), cos(inputs[:, 2])], axis=1)))
  elif dataset == 'r5':
      model.add(Lambda(lambda inputs: tf.stack([inputs[:, 0], inputs[:, 1], inputs[:, 2], inputs[:, 3], inputs[:, 4],
                                                  sin(inputs[:, 0]), sin(inputs[:, 1]), sin(inputs[:, 2]), sin(inputs[:, 3]), sin(inputs[:, 4]),
                                                  cos(inputs[:, 0]), cos(inputs[:, 1]), cos(inputs[:, 2]), cos(inputs[:, 3]), cos(inputs[:, 4])], axis=1)))

if dataset == 'r5':
  for units in r5_units:
    model.add(Dense(units=units, activation='relu'))
    hidden_layers = len(r5_units)
else:
  for _ in range(hidden_layers):
    model.add(Dense(units_per_layer, activation='relu'))

if dropout > 0:
  model.add(Dropout(dropout))
model.add(Dense(output_dim))


model.compile(optimizer=optimizer,
              loss=loss,
              metrics=metrics)

#### Training

In [ ]:
start_log_wandb(learning_rate, type(model).__name__, dataset, epochs, opt, loss, metrics, batch_size, split, hidden_layers, units_per_layer, dropout, full_output, noise, noise_percent)


In [ ]:
callbacks = [
    WandbCallback(
        monitor="val_mean_squared_error",
        mode="min",
        log_weights=False,
        log_gradients=False,
        save_graph=False
    )
]

In [ ]:
history = model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, validation_split=0.2, verbose=1, callbacks=callbacks)
test_loss, test_mae = model.evaluate(X_test, y_test, verbose=0)
print(f'Test MAE: {test_mae:.4f}')
wandb.log({
    "test_loss": test_loss,
    "test_mae": test_mae
})

#### Jacobian comparison

In [ ]:
@tf.function
def FK_Jacobian(model, x):
    x = tf.reshape(x, (1, x.shape[0]))
    with tf.GradientTape(persistent=True) as tape:
        tape.watch(x)
        y = model(x)
    jacobian_matrix = tape.jacobian(y, x)
    return jacobian_matrix

In [ ]:
def analytical_jacobian_r2(theta):
    theta = theta.flatten()
    l1 = 0.1
    l2 = 0.1
    jacobian = np.zeros((2, 2))
    jacobian[0, 0] = -l1 * np.sin(theta[0]) - l2 * np.sin(theta[0] + theta[1])
    jacobian[0, 1] = -l2 * np.sin(theta[0] + theta[1])
    jacobian[1, 0] = l1 * np.cos(theta[0]) + l2 * np.cos(theta[0] + theta[1])
    jacobian[1, 1] = l2 * np.cos(theta[0] + theta[1])
    return jacobian

def analytical_jacobian_r3(theta):

    theta = theta.flatten()
    if len(theta) < 3:
        raise ValueError("Input theta must have at least 3 elements.")

    l1 = 0.1
    l2 = 0.1
    l3 = 0.1
    jacobian = np.zeros((2, 3))

    jacobian[0, 0] = -l1 * np.sin(theta[0]) - l2 * np.sin(theta[0] + theta[1]) - l3 * np.sin(theta[0] + theta[1] + theta[2])
    jacobian[0, 1] = -l2 * np.sin(theta[0] + theta[1]) - l3 * np.sin(theta[0] + theta[1] + theta[2])
    jacobian[0, 2] = -l3 * np.sin(theta[0] + theta[1] + theta[2])

    jacobian[1, 0] = l1 * np.cos(theta[0]) + l2 * np.cos(theta[0] + theta[1]) + l3 * np.cos(theta[0] + theta[1] + theta[2])
    jacobian[1, 1] = l2 * np.cos(theta[0] + theta[1]) + l3 * np.cos(theta[0] + theta[1] + theta[2])
    jacobian[1, 2] = l3 * np.cos(theta[0] + theta[1] + theta[2])

    return jacobian

def analytical_jacobian_r5(theta):
    if len(theta) != 5:
        raise ValueError("Input theta must have 5 elements.")

    a = [0.0, 0.45, 0.11, 0.11, 0.115]

    def transformation_matrix(theta, alpha, a, d):
        return np.array([
            [np.cos(theta), -np.sin(theta) * np.cos(alpha), np.sin(theta) * np.sin(alpha), a * np.cos(theta)],
            [np.sin(theta), np.cos(theta) * np.cos(alpha), -np.cos(theta) * np.sin(alpha), a * np.sin(theta)],
            [0, np.sin(alpha), np.cos(alpha), d],
            [0, 0, 0, 1]
        ])

    T01 = np.array([
        [np.sin(theta[0]), 0, np.cos(theta[0]), a[0] * np.sin(theta[0])],
        [np.cos(theta[0]), 0, np.sin(theta[0]), -a[0] * np.cos(theta[0])],
        [0, -1, 0, 0],
        [0, 0, 0, 1]
    ])

    T12 = np.array([
        [np.sin(theta[1]), np.cos(theta[1]), 0, a[1] * np.sin(theta[1])],
        [-np.cos(theta[1]), np.sin(theta[1]), 0, -a[1] * np.cos(theta[1])],
        [0, 0, 1, 0],
        [0, 0, 0, 1]
    ])

    T23 = np.array([
        [np.cos(theta[2]), -np.sin(theta[2]), 0, a[2] * np.cos(theta[2])],
        [np.sin(theta[2]), np.cos(theta[2]), 0, a[2] * np.sin(theta[2])],
        [0, 0, 1, 0],
        [0, 0, 0, 1]
    ])

    T34 = np.array([
        [np.cos(theta[3]), 0, -np.sin(theta[3]), a[3] * np.cos(theta[3])],
        [np.sin(theta[3]), 0, np.cos(theta[3]), a[3] * np.sin(theta[3])],
        [0, -1, 0, 0],
        [0, 0, 0, 1]
    ])

    T45 = np.array([
        [np.cos(theta[4]), np.sin(theta[4]), 0, a[4] * np.cos(theta[4])],
        [np.sin(theta[4]), -np.cos(theta[4]), 0, a[4] * np.sin(theta[4])],
        [0, 0, -1, 0],
        [0, 0, 0, 1]
    ])

    T05 = T01 @ T12 @ T23 @ T34 @ T45
    px, py, pz = T05[0, 3], T05[1, 3], T05[2, 3]

    J_pos = np.zeros((3, 5))
    for i in range(5):
        theta_copy = theta.copy()
        d_theta = 1e-5
        theta_copy[i] += d_theta

        T01_new = np.array([
            [np.sin(theta_copy[0]), 0, np.cos(theta_copy[0]), a[0] * np.sin(theta_copy[0])],
            [np.cos(theta_copy[0]), 0, np.sin(theta_copy[0]), -a[0] * np.cos(theta_copy[0])],
            [0, -1, 0, 0],
            [0, 0, 0, 1]
        ])

        T12_new = np.array([
            [np.sin(theta_copy[1]), np.cos(theta_copy[1]), 0, a[1] * np.sin(theta_copy[1])],
            [-np.cos(theta_copy[1]), np.sin(theta_copy[1]), 0, -a[1] * np.cos(theta_copy[1])],
            [0, 0, 1, 0],
            [0, 0, 0, 1]
        ])

        T23_new = np.array([
            [np.cos(theta_copy[2]), -np.sin(theta_copy[2]), 0, a[2] * np.cos(theta_copy[2])],
            [np.sin(theta_copy[2]), np.cos(theta_copy[2]), 0, a[2] * np.sin(theta_copy[2])],
            [0, 0, 1, 0],
            [0, 0, 0, 1]
        ])

        T34_new = np.array([
            [np.cos(theta_copy[3]), 0, -np.sin(theta_copy[3]), a[3] * np.cos(theta_copy[3])],
            [np.sin(theta_copy[3]), 0, np.cos(theta_copy[3]), a[3] * np.sin(theta_copy[3])],
            [0, -1, 0, 0],
            [0, 0, 0, 1]
        ])

        T45_new = np.array([
            [np.cos(theta_copy[4]), np.sin(theta_copy[4]), 0, a[4] * np.cos(theta_copy[4])],
            [np.sin(theta_copy[4]), -np.cos(theta_copy[4]), 0, a[4] * np.sin(theta_copy[4])],
            [0, 0, -1, 0],
            [0, 0, 0, 1]
        ])

        T05_new = T01_new @ T12_new @ T23_new @ T34_new @ T45_new
        px_new, py_new, pz_new = T05_new[0, 3], T05_new[1, 3], T05_new[2, 3]

        J_pos[0, i] = (px_new - px) / d_theta
        J_pos[1, i] = (py_new - py) / d_theta
        J_pos[2, i] = (pz_new - pz) / d_theta

    return J_pos

In [ ]:
X = np.array(X_train[1])
X_model = inputs = tf.expand_dims(tf.convert_to_tensor(X, dtype=tf.float32), axis=0)

if dataset == 'r2':
  jacobian_matrix_true = analytical_jacobian_r2(X)
  jacobian_matrix_true = tf.squeeze(jacobian_matrix_true).numpy()
elif dataset == 'r3':
  jacobian_matrix_true = analytical_jacobian_r3(X)
  jacobian_matrix_true = tf.squeeze(jacobian_matrix_true).numpy()
elif dataset == 'r5':
  jacobian_matrix_true = analytical_jacobian_r5(X)
  jacobian_matrix_true = tf.squeeze(jacobian_matrix_true).numpy()
else:
  raise ValueError(f"Unknown dataset: {dataset}")

tf.compat.v1.disable_eager_execution()
jacobian_matrix_model = FK_Jacobian(model, X_model)
jacobian_matrix_model = tf.squeeze(jacobian_matrix_model).numpy()

print(jacobian_matrix_true)
print(jacobian_matrix_model)
mse_jacobian = np.mean((jacobian_matrix_true - jacobian_matrix_model) ** 2)
print(mse_jacobian)

wandb.log({"mse_jacobian": mse_jacobian})

In [ ]:
model_name = wandb.run.name + '_' +dataset + '_' + type(model).__name__ + str(hidden_layers)  + '_' + loss + '_' + opt + '_' + str(learning_rate)
model_file = model_name + '.h5'
print(model_file)
model.save(model_file)

wandb.save(model_file)

artifact = wandb.Artifact(model_name, type="model")
artifact.add_file(model_file)
wandb.log_artifact(artifact)

wandb.finish()

#### Hyperparameter search

In [ ]:
input_dim = X_train.shape[1]
output_dim = y_train.shape[1]

def build_model(hp):
    model = Sequential()
    model.add(InputLayer(shape=(input_dim,)))
    for i in range(hp.Int('hidden_layers', 1, 10)):
        units = hp.Int(f'units_{i}', min_value=32, max_value=256, step=32)
        model.add(Dense(units=units, activation='relu'))

    model.add(Dense(output_dim))

    optimizer_choice = hp.Choice('optimizer', values=['adam', 'sgd', 'rmsprop'])

    if optimizer_choice == 'adam':
        optimizer = Adam(learning_rate=hp.Float('learning_rate', 1e-4, 1e-1, sampling='log'))
    elif optimizer_choice == 'sgd':
        optimizer = SGD(learning_rate=hp.Float('learning_rate', 1e-4, 1e-1, sampling='log'))
    elif optimizer_choice == 'rmsprop':
        optimizer = RMSprop(learning_rate=hp.Float('learning_rate', 1e-4, 1e-1, sampling='log'))

    loss_choice = hp.Choice('loss', values=['mse', 'mae', 'huber', 'logcosh'])

    if loss_choice == 'mse':
        loss = MeanSquaredError()
    elif loss_choice == 'mae':
        loss = MeanAbsoluteError()
    elif loss_choice == 'huber':
        loss = Huber()
    elif loss_choice == 'logcosh':
        loss = LogCosh()

    model.compile(optimizer=optimizer,
                  loss=loss,
                  metrics=[MeanAbsoluteError()])

    return model

tuner = Hyperband(
    build_model,
    objective='val_mean_absolute_error',
    max_epochs=50,
    factor=3,
    directory=base_path + 'hyperband_dir',
    project_name='ann_hyperband_tuning' + dataset,
    executions_per_trial=1
)

In [ ]:
tuner.search(X_train, y_train, epochs=50, validation_data=(X_test, y_test))

In [ ]:
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]
hidden_units = []
print("Best hyperparameters:")
print(f"Hidden Layers: {best_hps.get('hidden_layers')}")
for i in range(10):
    units = best_hps.get(f'units_{i}')
    if i < 8:
        hidden_units.append(units)
print(f"Learning Rate: {best_hps.get('learning_rate')}")
print(f"Optimizer: {best_hps.get('optimizer')}")
print(f"Loss: {best_hps.get('loss')}")
print(f"Hidden units per layer: {hidden_units}")

In [ ]:
model = tuner.get_best_models(num_models=1)[0]
model.summary()
test_loss, test_mae = model.evaluate(X_test, y_test, verbose=0)
print(f'Test MAE: {test_mae:.4f}')
print(f'Test Loss: {test_loss:.6f}')

In [ ]:
best_model = tuner.get_best_models(num_models=1)[0]
best_model.save(f"{base_path}/models/{dataset}_tuned.h5")

#### SVR



In [ ]:
svr = MultiOutputRegressor(SVR(kernel='rbf', gamma='auto', epsilon=0.01, C=10))

In [ ]:
svr.fit(X_train, y_train)
y_pred = svr.predict(X_test)

mae_x = mean_absolute_error(y_test[:, 0], y_pred[:, 0])
mae_y = mean_absolute_error(y_test[:, 1], y_pred[:, 1])
total_mae = (mae_x + mae_y) / 2
print(f"Best MAE for x prediction: {mae_x}")
print(f"Best MAE for y prediction: {mae_y}")
print(f"Best Total MAE: {total_mae}")

Best MAE for x prediction: 0.005642613406925205
Best MAE for y prediction: 0.005780416908938051
Best Total MAE: 0.005711515157931628


In [ ]:
param_grid = {
    "estimator__C": [0.1, 1, 10, 100],
    "estimator__epsilon": [0.01, 0.1, 0.2, 0.5],
    "estimator__gamma": ["scale", "auto"]
}

svr = MultiOutputRegressor(SVR(kernel='rbf'))
grid_search = RandomizedSearchCV(svr, param_grid, n_iter=10, cv=3, n_jobs=-1, verbose=3)

grid_search.fit(X_train, y_train)

best_model = grid_search.best_estimator_

y_pred = best_model.predict(X_test)

mae_x = mean_absolute_error(y_test[:, 0], y_pred[:, 0])
mae_y = mean_absolute_error(y_test[:, 1], y_pred[:, 1])
total_mae = (mae_x + mae_y) / 2

print("Best parameters found:", grid_search.best_params_)
print(f"Best MAE for x prediction: {mae_x}")
print(f"Best MAE for y prediction: {mae_y}")
print(f"Best Total MAE: {total_mae}")

#### Model testing

In [ ]:
model = load_model(base_path + 'models/r5_tuned.h5')
model.summary()

test_loss, test_mae = model.evaluate(X_test, y_test, verbose=0)
print(f'Test Loss: {test_loss}')
print(f'Test MAE: {test_mae:.4f}')

In [ ]:
run = wandb.init(project="MLH1")
artifact = run.use_artifact('sapienza-ceriotti-hw/MLH1/charmed-wave-40_r5_Sequential8_logCosH_adam_0.00025:v0', type='model')
artifact_dir = artifact.download()

In [ ]:
model = load_model(artifact_dir + '/charmed-wave-40_r5_Sequential8_logCosH_adam_0.00025.h5', compile=False)
model.summary()

In [ ]:
# Erase the weights
model_config = model.to_json()

new_model = tf.keras.models.model_from_json(model_config)
model = new_model

In [ ]:
opt = 'adam'
loss = 'mse'
model.compile(optimizer=opt,
                  loss=loss,
                  metrics=[MeanAbsoluteError()])

In [ ]:
test_loss, test_mae = model.evaluate(X_test, y_test, verbose=0)
print(f'Test MAE: {test_mae:.4f}')
print(f"Test Loss: {test_loss:.6f}")

In [ ]:
optimizer = model.optimizer
print(f"Optimizer: {optimizer}")

learning_rate = optimizer.learning_rate
print(f"Learning Rate: {learning_rate.numpy()}")

loss = model.loss
print(f"Loss: {loss}")

metrics = model.metrics_names
print(f"Metrics: {metrics}")

## Inverse Kinematics

In [ ]:
def inverse_kinematics_r2(model, x_d, initial_guess=np.array([-0.5, 0.5]),
                                 max_iters=500, tolerance=1e-5, damping=0.01):
    theta = initial_guess

    for i in range(max_iters):

        x_pred = model(tf.reshape(theta, (1, 2)))
        x_pred = x_pred.numpy().flatten()

        error = np.array([x_pred[0] - x_d[0], x_pred[1] - x_d[1]])

        if np.linalg.norm(error) < tolerance:
            print(f"Converged after {i+1} iterations: {theta}")
            return theta

        jacobian_matrix = FK_Jacobian(model, theta)
        jacobian_matrix = jacobian_matrix.numpy().reshape(2, 2)

        JTJ = jacobian_matrix.T @ jacobian_matrix
        identity = damping * np.eye(JTJ.shape[0])
        delta_theta = np.linalg.solve(JTJ + identity, -jacobian_matrix.T @ error)

        theta = theta + delta_theta
        theta = (theta + np.pi) % (2 * np.pi) - np.pi

    print("Did not converge within the maximum number of iterations")
    return theta

def inverse_kinematics_r3(model, x_d, initial_guess=np.array([-0.5, 0.5, 0.5]),
                                 max_iters=500, tolerance=1e-5, damping=0.01):
    theta = initial_guess

    for i in range(max_iters):

        x_pred = model(tf.reshape(theta, (1, 3)))
        x_pred = x_pred.numpy().flatten()

        error = np.array([x_pred[0] - x_d[0], x_pred[1] - x_d[1]])

        if np.linalg.norm(error) < tolerance:
            print(f"Converged after {i+1} iterations: {theta}")
            return theta

        jacobian_matrix = FK_Jacobian(model, theta)
        jacobian_matrix = jacobian_matrix.numpy().reshape(2, 3)

        JTJ = jacobian_matrix.T @ jacobian_matrix
        identity = damping * np.eye(JTJ.shape[0])
        delta_theta = np.linalg.solve(JTJ + identity, -jacobian_matrix.T @ error)

        theta = theta + delta_theta
        theta = (theta + np.pi) % (2 * np.pi) - np.pi

    print("Did not converge within the maximum number of iterations")
    return theta


def inverse_kinematics_r5(model, x_d, initial_guess=np.array([-0.5, 0.5, 0.5, 0.5, 0.5]),
                                 max_iters=500, tolerance=1e-5, damping=0.01):
    theta = initial_guess

    for i in range(max_iters):

        x_pred = model(tf.reshape(theta, (1, 5)))
        x_pred = x_pred.numpy().flatten()

        error = np.array([x_pred[0] - x_d[0], x_pred[1] - x_d[1], x_pred[2] - x_d[2]])

        if np.linalg.norm(error) < tolerance:
            print(f"Converged after {i+1} iterations: {theta}")
            return theta

        jacobian_matrix = FK_Jacobian(model, theta)
        jacobian_matrix = jacobian_matrix.numpy().reshape(3, 5)

        JTJ = jacobian_matrix.T @ jacobian_matrix
        identity = damping * np.eye(JTJ.shape[0])
        delta_theta = np.linalg.solve(JTJ + identity, -jacobian_matrix.T @ error)

        theta = theta + delta_theta
        theta = (theta + np.pi) % (2 * np.pi) - np.pi

    print("Did not converge within the maximum number of iterations")
    return theta



In [ ]:
random_idx = np.random.randint(len(X_test))
x_d = y_test[random_idx].numpy()
true_joint_angles = X_test[random_idx].numpy()
if dataset == 'r2':
  theta_solution = inverse_kinematics_r2(model, x_d)
if dataset == 'r3':
  theta_solution = inverse_kinematics_r3(model, x_d)
if dataset == 'r5':
  theta_solution = inverse_kinematics_r5(model, x_d)

theta_solution = (theta_solution + np.pi) % (2 * np.pi) - np.pi
true_joint_angles = (true_joint_angles + np.pi) % (2 * np.pi) - np.pi

predicted_position = model(tf.reshape(theta_solution, (1, theta_solution.shape[0]))).numpy().flatten()
position_mae = mean_absolute_error(x_d, predicted_position)

print(f"Solution for the joint angles: {theta_solution}")
print(f"True joint angles: {true_joint_angles}")
print(f"Position-based MAE between predicted and true positions: {position_mae:.6f}")